# This file explores, preprocesses and analyzes the Census Data
##### Data primarily consists of housing statistics and estimates 

In [1]:
import pandas as pd
import re

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

house_df = pd.read_csv("../Data/houseInfoMorris.csv")

In [2]:
house_df.columns

Index(['Label (Grouping)', 'Boonton town, Morris County, New Jersey!!Estimate',
       'Boonton town, Morris County, New Jersey!!Margin of Error',
       'Boonton town, Morris County, New Jersey!!Percent',
       'Boonton town, Morris County, New Jersey!!Percent Margin of Error',
       'Boonton township, Morris County, New Jersey!!Estimate',
       'Boonton township, Morris County, New Jersey!!Margin of Error',
       'Boonton township, Morris County, New Jersey!!Percent',
       'Boonton township, Morris County, New Jersey!!Percent Margin of Error',
       'Butler borough, Morris County, New Jersey!!Estimate',
       ...
       'Victory Gardens borough, Morris County, New Jersey!!Percent',
       'Victory Gardens borough, Morris County, New Jersey!!Percent Margin of Error',
       'Washington township, Morris County, New Jersey!!Estimate',
       'Washington township, Morris County, New Jersey!!Margin of Error',
       'Washington township, Morris County, New Jersey!!Percent',
      

## 1.) Compute Average Margin of Error

In [126]:
import numpy as np
desired_columns1 = [x for x in house_df.columns if re.search(r"!!Margin of Error$", x)]
margin_df = house_df.filter(items=["Label (Grouping)"]+desired_columns1)
desired_columns = [x for x in house_df.columns if re.search(r"!!Estimate$", x)]
est = house_df.filter(items=["Label (Grouping)"]+desired_columns)
est = est.replace(',','', regex=True).replace('±','', regex=True).replace('\\+', '0', regex=True).replace('\\*', '0', regex=True)
margin_df = margin_df.replace(',','', regex=True).replace('±','', regex=True).replace('\\+', '0', regex=True).replace('\\*', '0', regex=True)

In [234]:
newdf = margin_df.copy()["Label (Grouping)"]
col_est = est.columns[1:]
col_margin = margin_df.columns[1:]
for x in range(39):
    if 0 in [float(y) for y in margin_df[col_margin[x]]]:
        newdf[x] = np.zeros(len(est))
    else:
        w = [float(est[col_est[x]][y])/float(margin_df[col_margin[x]][y]) for y in range(len(est))]
        newdf[x] = w

In [235]:
newdf = newdf[:39]
k = [pd.DataFrame(newdf.iloc[x]) for x in range(len(newdf))]
k = [x.replace(['nan',float('NaN')], 0) for x in k]

In [254]:
k_mean = [np.mean(x) for x in k]
k_mean = np.mean(k_mean)
k_mean

3.0593134280736254

## 2.) Keep columns containing estimates

In [167]:
# Need to keep columns that only have estimates 
desired_columns = [x for x in house_df.columns if re.search(r"!!Estimate$", x)]
house_sub_df = house_df.filter(items=["Label (Grouping)"]+desired_columns)
house_sub_df.head()

,Label (Grouping),"Boonton town, Morris County, New Jersey!!Estimate","Boonton township, Morris County, New Jersey!!Estimate","Butler borough, Morris County, New Jersey!!Estimate","Chatham borough, Morris County, New Jersey!!Estimate","Chatham township, Morris County, New Jersey!!Estimate","Chester borough, Morris County, New Jersey!!Estimate","Chester township, Morris County, New Jersey!!Estimate","Denville township, Morris County, New Jersey!!Estimate","Dover town, Morris County, New Jersey!!Estimate",...,"Parsippany-Troy Hills township, Morris County, New Jersey!!Estimate","Pequannock township, Morris County, New Jersey!!Estimate","Randolph township, Morris County, New Jersey!!Estimate","Riverdale borough, Morris County, New Jersey!!Estimate","Rockaway borough, Morris County, New Jersey!!Estimate","Rockaway township, Morris County, New Jersey!!Estimate","Roxbury township, Morris County, New Jersey!!Estimate","Victory Gardens borough, Morris County, New Jersey!!Estimate","Washington township, Morris County, New Jersey!!Estimate","Wharton borough, Morris County, New Jersey!!Estimate"
0,HOUSING OCCUPANCY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total housing units,"3,639","1,532","3,523","3,058","3,939",596,"2,680","6,970","6,379",...,"22,746","6,442","9,340","1,954","2,674","9,906","8,593",704,"6,571","2,913"
2,Occupied housing units,"3,414","1,411","3,482","2,950","3,849",582,"2,542","6,425","6,128",...,"22,159","6,210","9,159","1,904","2,641","9,163","8,163",670,"6,352","2,779"
3,Vacant housing units,225,121,41,108,90,14,138,545,251,...,587,232,181,50,33,743,430,34,219,134
4,Homeowner vacancy rate,0.0,3.1,0.0,0.0,2.4,0.0,4.2,0.8,2.0,...,0.5,0.0,0.5,0.4,0.0,2.4,0.3,3.7,0.6,0.0


## 3.) Preprocess Dataframe

In [168]:
house_sub_df.dropna(inplace=True) # Drop NAs (really just row headers)
house_sub_df.drop_duplicates(inplace=True) # Drop duplicates 
new_column_names = [x.split(',')[0] for x in house_sub_df.columns[1:]]
house_sub_df.columns = ["Information"]+new_column_names # Replace columns with just neighbourhood names
house_sub_df = house_sub_df.T # Transpose
house_sub_df.columns = house_sub_df.iloc[0] # Make first row as column headers
house_sub_df.drop(house_sub_df.index[0], axis=0, inplace=True) # drop first row
new_column_names = [x.replace(' ', '') for x in house_sub_df.columns] 
house_sub_df.columns = new_column_names # Replace columns without weird whitespace

## 4.) Keep relevant columns/features 

In [169]:
# Need to do research on what features I should remove
# So far, we have when owner moved in, number of vehicles, gross rent, renters versus owner occupied, year built,
# Note that median (dollars has 4 different purposes)

desired_features = ["Moved in 2021 or later","Moved in 2018 to 2020","Moved in 2010 to 2017","Moved in 2000 to 2009","Moved in 1990 to 1999","Moved in 1989 and earlier","No vehicles available","1 vehicle available","2 vehicles available","3 or more vehicles available","Median (dollars)","Occupied housing units","Owner-occupied","Renter-occupied","Built 2020 or later","Built 2010 to 2019","Built 2000 to 2009","Built 1990 to 1999","Built 1980 to 1989","Built 1970 to 1979","Built 1960 to 1969","Built 1950 to 1959","Built 1940 to 1949","Built 1939 or earlier"]

house_sub_df = house_sub_df[desired_features]

In [170]:
# First median == Value
# Second median == Monthly Ownership costs
# Third median == Monthly Ownership costs 2
# Fourth median == Gross Rent

new_column_names = []
cnt = 0
for col in house_sub_df.columns:
    if col == "Median (dollars)":
        if cnt == 0:
            new_column_names.append("Median Home Value")
        elif cnt == 3 :
            new_column_names.append("Median Gross Rent")
        else:
            new_column_names.append("Drop")
        cnt += 1
    else:
        new_column_names.append(col)
    
house_sub_df.columns = new_column_names
house_sub_cleaned = house_sub_df.drop("Drop", axis=1)

In [171]:
house_sub_cleaned.reset_index(inplace=True)
house_sub_cleaned.rename(columns={"index": "town"}, inplace=True)

## 5.) Need to rename town names to match the official Morris County Registry

In [172]:
change_dict = {'Boonton town':'Boonton Town','Boonton township':'Boonton Township','Butler borough':'Butler Borough','Chatham borough':'Chatham Borough','Chatham township':'Chatham Township','Chester borough':'Chester Borough','Chester township':'Chester Township','Denville township':'Denville Township','Dover town':'Dover Town','East Hanover township':'East Hanover Township','Florham Park borough':'Florham Park Borough','Hanover township':'Hanover Township','Harding township':'Harding Township','Jefferson township':'Jefferson Township','Kinnelon borough':'Kinnelon Borough','Lincoln Park borough':'Lincoln Park Borough','Long Hill township':'Long Hill Township','Madison borough':'Madison Borough','Mendham borough':'Mendham Borough','Mendham township':'Mendham Township','Mine Hill township':'Mine Hill Township','Montville township':'Montville Township','Morris township':'Morris Township','Morris Plains borough':'Morris Plains Borough','Morristown town':'Morristown Town','Mountain Lakes borough':'Mountain Lakes Borough','Mount Arlington borough':'Mount Arlington Borough','Mount Olive township':'Mount Olive Township','Netcong borough':'Netcong Borough','Parsippany-Troy Hills township':'Parsippany-Troy Hills Township','Pequannock township':'Pequannock Township','Randolph township':'Randolph Township','Riverdale borough':'Riverdale Borough','Rockaway borough':'Rockaway Borough','Rockaway township':'Rockaway Township','Roxbury township':'Roxbury Township','Victory Gardens borough':'Victory Gardens Borough','Washington township':'Washington Township','Wharton borough':'Wharton Borough'}

house_sub_cleaned.replace({'town': change_dict}, inplace=True)

## 6.) Remove some data for better model

In [173]:
house_sub_cleaned.columns

Index(['town', 'Moved in 2021 or later', 'Moved in 2018 to 2020',
       'Moved in 2010 to 2017', 'Moved in 2000 to 2009',
       'Moved in 1990 to 1999', 'Moved in 1989 and earlier',
       'No vehicles available', '1 vehicle available', '2 vehicles available',
       '3 or more vehicles available', 'Median Home Value',
       'Median Gross Rent', 'Occupied housing units', 'Occupied housing units',
       'Owner-occupied', 'Renter-occupied', 'Built 2020 or later',
       'Built 2010 to 2019', 'Built 2000 to 2009', 'Built 1990 to 1999',
       'Built 1980 to 1989', 'Built 1970 to 1979', 'Built 1960 to 1969',
       'Built 1950 to 1959', 'Built 1940 to 1949', 'Built 1939 or earlier'],
      dtype='object')

In [174]:
house_sub_cleaned.drop(['Moved in 2021 or later', 'Moved in 2018 to 2020',
       'Moved in 2010 to 2017', 'Moved in 2000 to 2009',
       'Moved in 1990 to 1999', 'Moved in 1989 and earlier',
       'No vehicles available', '1 vehicle available', '2 vehicles available',
       '3 or more vehicles available'],axis=1, inplace=True)
house_sub_cleaned = house_sub_cleaned.T.drop_duplicates().T
house_sub_cleaned.columns

Index(['town', 'Median Home Value', 'Median Gross Rent',
       'Occupied housing units', 'Owner-occupied', 'Renter-occupied',
       'Built 2020 or later', 'Built 2010 to 2019', 'Built 2000 to 2009',
       'Built 1990 to 1999', 'Built 1980 to 1989', 'Built 1970 to 1979',
       'Built 1960 to 1969', 'Built 1950 to 1959', 'Built 1940 to 1949',
       'Built 1939 or earlier'],
      dtype='object')

In [175]:
house_sub_cleaned.to_csv("censusMorrisCleaned.csv", index=False)